In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import json
from bs4 import BeautifulSoup
import urllib3

In [7]:
http = urllib3.PoolManager()
response = http.request('GET', 'https://s3.amazonaws.com/csye7374/config.json')
doc = str(BeautifulSoup(response.data))

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
data = json.loads(doc)

In [9]:
data['Activation'][1]

'elu'

In [10]:
batch_size = data['fit']['Batch_Size']
print(batch_size)
num_classes = data['fit']['Num_Classes']
print(num_classes)
epochs = data['fit']['epochs']
print(epochs)
data_augmentation = data['fit']['Data_Augmentation']
print(data_augmentation)
lr=data['compile']['Learning_Rate']
print(lr)
decay=data['compile']['Decay']
print(decay)

32
10
60
False
0.0001
1e-06


In [11]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 107s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=decay)

In [0]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [16]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=data['fit']['Shuffle'])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/60
50000/50000 [==============================] - 27s 539us/step - loss: 1.7947 - acc: 0.3415 - val_loss: 1.5214 - val_acc: 0.4447
Epoch 2/60
50000/50000 [==============================] - 23s 462us/step - loss: 1.4598 - acc: 0.4703 - val_loss: 1.3584 - val_acc: 0.5118
Epoch 3/60
50000/50000 [==============================] - 23s 460us/step - loss: 1.3250 - acc: 0.5292 - val_loss: 1.2859 - val_acc: 0.5419
Epoch 4/60
50000/50000 [==============================] - 23s 460us/step - loss: 1.2313 - acc: 0.5644 - val_loss: 1.1992 - val_acc: 0.5759
Epoch 5/60
50000/50000 [==============================] - 23s 459us/step - loss: 1.1579 - acc: 0.5916 - val_loss: 1.1207 - val_acc: 0.6045
Epoch 6/60
50000/50000 [==============================] - 23s 463us/step - loss: 1.0929 - acc: 0.6157 - val_loss: 1.0676 - val_acc: 0.6258
Epoch 7/60
50000/50000 [==============================] - 23s 459us/step - loss: 1.0377

In [0]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model1.add(Activation('elu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('elu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), padding='same'))
model1.add(Activation('elu'))
model1.add(Conv2D(128, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('elu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [0]:
data_augmentation = True

In [0]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
if not data_augmentation:
    print('Not using data augmentation.')
    model1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=data['fit']['Shuffle'])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model1.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/60
1563/1563 [==============================] - 57s 36ms/step - loss: 1.6414 - acc: 0.4112 - val_loss: 1.2490 - val_acc: 0.5489
Epoch 2/60
1563/1563 [==============================] - 56s 36ms/step - loss: 1.4020 - acc: 0.5078 - val_loss: 1.1355 - val_acc: 0.6009
Epoch 3/60
1563/1563 [==============================] - 55s 35ms/step - loss: 1.3336 - acc: 0.5374 - val_loss: 1.0697 - val_acc: 0.6332
Epoch 4/60
1563/1563 [==============================] - 54s 35ms/step - loss: 1.3077 - acc: 0.5470 - val_loss: 0.9014 - val_acc: 0.6826
Epoch 5/60
1563/1563 [==============================] - 56s 36ms/step - loss: 1.2935 - acc: 0.5592 - val_loss: 1.0146 - val_acc: 0.6487
Epoch 6/60
1563/1563 [==============================] - 56s 36ms/step - loss: 1.2771 - acc: 0.5640 - val_loss: 0.9215 - val_acc: 0.6867
Epoch 7/60
1563/1563 [==============================] - 55s 35ms/step - loss: 1.2741 - acc: 0.5689 - val_loss: 1.0635 - val_acc: 0.6446
Epoch 8/60
15

KeyboardInterrupt: ignored